##### 测试KITree不同超参的效果，使用'cicids_custom'和'toniot_custom'数据集，使用"AE","VAE","Whisper",'Whisper',"IForest","OCSVM"作为black_model

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../../src/')
import numpy as np
import pickle
import torch
import torch.nn as nn
from global_var import *
from normalize import *
from data_load import *
from utils import *
from AE import AutoEncoder
from VAE import VAE
import ExtBound
import KITree
import importlib
from IBMMExtraction import *
import utils as Utils
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from utils import evaluate_and_save_results_Hyperparameters

importlib.reload(KITree)
models = ["AE","VAE","Whisper","OCSVM","IForest"] 

datasets = {
    'toniot_custom': [
        'backdoor',
        'ddos',
        'dos',
        'injection',
        'mitm',
        'password',
        'runsomware',
        'scanning',
        'xss'
    ]
}

max_levels = [5, 10, 15, 20, 25]
n_beams = [2, 4, 6, 8, 10]
rhos = [0.01, 0.05, 0.1, 0.5, 1]
etas = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

In [ ]:
def loadData(dataset, subset):
    train_data, test_data, train_target, test_target = load_data(dataset, subset, 'train') 
    test_data, test_target = load_data(dataset, subset, mode='test')
    with open(os.path.join(NORMALIZER_DIR, f'{dataset}_{subset}.norm'), 'rb') as f:
        normalizer = pickle.load(f)
    train_data = normalizer.transform(train_data)
    test_data = normalizer.transform(test_data)
    return train_data, test_data, train_target, test_target

def perturb_data_point(data_point, delta):
    return data_point + delta

In [2]:
for max_level_ in max_levels:
    for model in models:
        for dataset in datasets:
            all_predictions = np.empty((0,))
            all_original_predictions = np.empty((0,))
            all_perturbed_predictions = np.empty((0,))
            all_y_test = np.empty((0,))

            for subset in datasets[dataset]:
                X_train, X_test, y_train, y_test = loadData(dataset, subset) 
                X, y = X_train, y_train

                if model == "VAE" or model == "AE":
                    blackbox_model = torch.load(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model')).cuda(DEVICE)
                    blackbox_model.eval()
                    func_ = lambda x: blackbox_model.score_samples(x)
                    score_ = func_(X)
                    thres_ = blackbox_model.thres
                else:
                    with open(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model'), 'rb') as f:
                        blackbox_model = pickle.load(f)
                    if model == "Whisper":
                        score_ = -blackbox_model.score_samples(X)
                        thres_ = blackbox_model.threshold
                        func_ = lambda x: -blackbox_model.score_samples(x)
                    else:
                        score_ = -blackbox_model.decision_function(X)
                        thres_ = -blackbox_model.offset_
                        func_ = lambda x: -blackbox_model.decision_function(x)

                kdt_ = KITree.KITree(func_, thres_,max_level=max_level_)
                kdt_.fit(X, score_)
                predictions = kdt_.predict(X_test)
                all_predictions = np.concatenate((all_predictions, predictions))

                original_predictions = blackbox_model.predict(X_test)
                if model == "IForest" or model == "OCSVM" or model == "Whisper":
                    original_predictions = np.where(original_predictions == 1, 0, original_predictions)
                    original_predictions = np.where(original_predictions == -1, 1, original_predictions)
                all_original_predictions = np.concatenate((all_original_predictions, original_predictions))

                def perturb_data_point(data_point, delta):
                    return np.array([x + delta for x in data_point]).reshape(1, -1)
                
                delta = 0.01 
                robustness_sum = 0
                perturbed_predictions = np.empty((0,))
                for i, data_point in enumerate(X_test):
                    perturbed_data_point = perturb_data_point(data_point, delta)
                    perturbed_prediction = kdt_.predict(perturbed_data_point)
                    if np.array_equal(predictions[i], perturbed_prediction):
                        robustness_sum += 1
                    perturbed_predictions = np.append(perturbed_predictions, perturbed_prediction)
                all_perturbed_predictions = np.concatenate((all_perturbed_predictions, perturbed_predictions))
                all_y_test = np.concatenate((all_y_test, y_test))

            print("================ Processing ( {dataset} ) dataset, using model = ( {model} )".format(dataset=dataset, model=model), "================")

            evaluate_and_save_results_Hyperparameters(all_y_test, all_predictions, all_original_predictions, all_perturbed_predictions, dataset, baseline="ours", black_model=model, max_level=max_level_)
print("============================= max_level All works are finished! =============================")

In [ ]:
for n_beam_ in n_beams:
    for model in models:
        for dataset in datasets:
            all_predictions = np.empty((0,))
            all_original_predictions = np.empty((0,))
            all_perturbed_predictions = np.empty((0,))
            all_y_test = np.empty((0,))

            for subset in datasets[dataset]:
                X_train, X_test, y_train, y_test = loadData(dataset, subset) 
                X, y = X_train, y_train

                if model == "VAE" or model == "AE":
                    blackbox_model = torch.load(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model')).cuda(DEVICE)
                    blackbox_model.eval()
                    func_ = lambda x: blackbox_model.score_samples(x)
                    score_ = func_(X)
                    thres_ = blackbox_model.thres
                else:
                    with open(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model'), 'rb') as f:
                        blackbox_model = pickle.load(f)
                    if model == "Whisper":
                        score_ = -blackbox_model.score_samples(X)
                        thres_ = blackbox_model.threshold
                        func_ = lambda x: -blackbox_model.score_samples(x)
                    else:
                        score_ = -blackbox_model.decision_function(X)
                        thres_ = -blackbox_model.offset_
                        func_ = lambda x: -blackbox_model.decision_function(x)

                kdt_ = KITree.KITree(func_, thres_,n_beam=n_beam_)
                kdt_.fit(X, score_)
                predictions = kdt_.predict(X_test)
                all_predictions = np.concatenate((all_predictions, predictions))

                original_predictions = blackbox_model.predict(X_test)
                if model == "IForest" or model == "OCSVM" or model == "Whisper":
                    original_predictions = np.where(original_predictions == 1, 0, original_predictions)
                    original_predictions = np.where(original_predictions == -1, 1, original_predictions)
                all_original_predictions = np.concatenate((all_original_predictions, original_predictions))

                def perturb_data_point(data_point, delta):
                    return np.array([x + delta for x in data_point]).reshape(1, -1)
                
                delta = 0.01  # Define perturbation magnitude
                robustness_sum = 0
                perturbed_predictions = np.empty((0,))
                for i, data_point in enumerate(X_test):
                    perturbed_data_point = perturb_data_point(data_point, delta)
                    perturbed_prediction = kdt_.predict(perturbed_data_point)
                    if np.array_equal(predictions[i], perturbed_prediction):
                        robustness_sum += 1
                    perturbed_predictions = np.append(perturbed_predictions, perturbed_prediction)
                all_perturbed_predictions = np.concatenate((all_perturbed_predictions, perturbed_predictions))
                all_y_test = np.concatenate((all_y_test, y_test))
            print("{dataset} _ y_test.shape = ".format(dataset=dataset))

            print("================ Processing ( {dataset} ) dataset, using model = ( {model} )".format(dataset=dataset, model=model), "================")

            evaluate_and_save_results_Hyperparameters(all_y_test, all_predictions, all_original_predictions, all_perturbed_predictions, dataset, baseline="ours", black_model=model, n_beam=n_beam_)
print("============================= n_beams All works are finished! =============================")

In [ ]:
for rho_ in rhos:
    for model in models:
        for dataset in datasets:
            all_predictions = np.empty((0,))
            all_original_predictions = np.empty((0,))
            all_perturbed_predictions = np.empty((0,))
            all_y_test = np.empty((0,))

            for subset in datasets[dataset]:
                X_train, X_test, y_train, y_test = loadData(dataset, subset) 
                X, y = X_train, y_train

                if model == "VAE" or model == "AE":
                    blackbox_model = torch.load(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model')).cuda(DEVICE)
                    blackbox_model.eval()
                    func_ = lambda x: blackbox_model.score_samples(x)
                    score_ = func_(X)
                    thres_ = blackbox_model.thres
                else:
                    with open(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model'), 'rb') as f:
                        blackbox_model = pickle.load(f)
                    if model == "Whisper":
                        score_ = -blackbox_model.score_samples(X)
                        thres_ = blackbox_model.threshold
                        func_ = lambda x: -blackbox_model.score_samples(x)
                    else:
                        score_ = -blackbox_model.decision_function(X)
                        thres_ = -blackbox_model.offset_
                        func_ = lambda x: -blackbox_model.decision_function(x)

                kdt_ = KITree.KITree(func_, thres_,rho=rho_)
                kdt_.fit(X, score_)
                predictions = kdt_.predict(X_test)
                all_predictions = np.concatenate((all_predictions, predictions))

                original_predictions = blackbox_model.predict(X_test)
                if model == "IForest" or model == "OCSVM" or model == "Whisper":
                    original_predictions = np.where(original_predictions == 1, 0, original_predictions)
                    original_predictions = np.where(original_predictions == -1, 1, original_predictions)
                all_original_predictions = np.concatenate((all_original_predictions, original_predictions))

                def perturb_data_point(data_point, delta):
                    return np.array([x + delta for x in data_point]).reshape(1, -1)
                
                delta = 0.01  # Define perturbation magnitude
                robustness_sum = 0
                perturbed_predictions = np.empty((0,))
                for i, data_point in enumerate(X_test):
                    perturbed_data_point = perturb_data_point(data_point, delta)
                    perturbed_prediction = kdt_.predict(perturbed_data_point)
                    if np.array_equal(predictions[i], perturbed_prediction):
                        robustness_sum += 1
                    perturbed_predictions = np.append(perturbed_predictions, perturbed_prediction)
                all_perturbed_predictions = np.concatenate((all_perturbed_predictions, perturbed_predictions))
                all_y_test = np.concatenate((all_y_test, y_test))
            print("{dataset} _ y_test.shape = ".format(dataset=dataset),all_y_test.shape)

            print("================ Processing ( {dataset} ) dataset, using model = ( {model} )".format(dataset=dataset, model=model), "================")


            evaluate_and_save_results_Hyperparameters(all_y_test, all_predictions, all_original_predictions, all_perturbed_predictions, dataset, baseline="ours", black_model=model, rho=rho_)
print("============================= rho All works are finished! =============================")

In [ ]:
for eta_ in etas:
    for model in models:
        for dataset in datasets:
            all_predictions = np.empty((0,))
            all_original_predictions = np.empty((0,))
            all_perturbed_predictions = np.empty((0,))
            all_y_test = np.empty((0,))

            for subset in datasets[dataset]:
                X_train, X_test, y_train, y_test = loadData(dataset, subset) 
                X, y = X_train, y_train

                if model == "VAE" or model == "AE":
                    blackbox_model = torch.load(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model')).cuda(DEVICE)
                    blackbox_model.eval()
                    func_ = lambda x: blackbox_model.score_samples(x)
                    score_ = func_(X)
                    thres_ = blackbox_model.thres
                else:
                    with open(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model'), 'rb') as f:
                        blackbox_model = pickle.load(f)
                    if model == "Whisper":
                        score_ = -blackbox_model.score_samples(X)
                        thres_ = blackbox_model.threshold
                        func_ = lambda x: -blackbox_model.score_samples(x)
                    else:
                        score_ = -blackbox_model.decision_function(X)
                        thres_ = -blackbox_model.offset_
                        func_ = lambda x: -blackbox_model.decision_function(x)

                kdt_ = KITree.KITree(func_, thres_,eta=eta_)
                kdt_.fit(X, score_)
                predictions = kdt_.predict(X_test)
                all_predictions = np.concatenate((all_predictions, predictions))

                original_predictions = blackbox_model.predict(X_test)
                if model == "IForest" or model == "OCSVM" or model == "Whisper":
                    original_predictions = np.where(original_predictions == 1, 0, original_predictions)
                    original_predictions = np.where(original_predictions == -1, 1, original_predictions)
                all_original_predictions = np.concatenate((all_original_predictions, original_predictions))

                def perturb_data_point(data_point, delta):
                    return np.array([x + delta for x in data_point]).reshape(1, -1)
                
                delta = 0.01  # Define perturbation magnitude
                robustness_sum = 0
                perturbed_predictions = np.empty((0,))
                for i, data_point in enumerate(X_test):
                    perturbed_data_point = perturb_data_point(data_point, delta)
                    perturbed_prediction = kdt_.predict(perturbed_data_point)
                    if np.array_equal(predictions[i], perturbed_prediction):
                        robustness_sum += 1
                    perturbed_predictions = np.append(perturbed_predictions, perturbed_prediction)
                all_perturbed_predictions = np.concatenate((all_perturbed_predictions, perturbed_predictions))
                all_y_test = np.concatenate((all_y_test, y_test))
            print("{dataset} _ y_test.shape = ".format(dataset=dataset),all_y_test.shape)

            print("================ Processing ( {dataset} ) dataset, using model = ( {model} )".format(dataset=dataset, model=model), "================")

            evaluate_and_save_results_Hyperparameters(all_y_test, all_predictions, all_original_predictions, all_perturbed_predictions, dataset, baseline="ours", black_model=model, eta=eta_)
print("============================= eta All works are finished! =============================")

In [ ]:
for rho_ in rhos:
    for model in models:
        for dataset in datasets:
            all_predictions = np.empty((0,))
            all_original_predictions = np.empty((0,))
            all_perturbed_predictions = np.empty((0,))
            all_y_test = np.empty((0,))

            for subset in datasets[dataset]:
                X_train, X_test, y_train, y_test = loadData(dataset, subset) 
                X, y = X_train, y_train

                if model == "VAE" or model == "AE":
                    blackbox_model = torch.load(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model')).cuda(DEVICE)
                    blackbox_model.eval()
                    func_ = lambda x: blackbox_model.score_samples(x)
                    score_ = func_(X)
                    thres_ = blackbox_model.thres
                else:
                    with open(os.path.join(TARGET_MODEL_DIR, f'{model}_{dataset}_{subset}.model'), 'rb') as f:
                        blackbox_model = pickle.load(f)
                    if model == "Whisper":
                        score_ = -blackbox_model.score_samples(X)
                        thres_ = blackbox_model.threshold
                        func_ = lambda x: -blackbox_model.score_samples(x)
                    else:
                        score_ = -blackbox_model.decision_function(X)
                        thres_ = -blackbox_model.offset_
                        func_ = lambda x: -blackbox_model.decision_function(x)

                kdt_ = KITree.KITree(func_, thres_,rho=rho_)
                kdt_.fit(X, score_)
                predictions = kdt_.predict(X_test)
                all_predictions = np.concatenate((all_predictions, predictions))

                original_predictions = blackbox_model.predict(X_test)
                if model == "IForest" or model == "OCSVM" or model == "Whisper":
                    original_predictions = np.where(original_predictions == 1, 0, original_predictions)
                    original_predictions = np.where(original_predictions == -1, 1, original_predictions)
                all_original_predictions = np.concatenate((all_original_predictions, original_predictions))

                def perturb_data_point(data_point, delta):
                    return np.array([x + delta for x in data_point]).reshape(1, -1)
                
                delta = 0.01  # Define perturbation magnitude
                robustness_sum = 0
                perturbed_predictions = np.empty((0,))
                for i, data_point in enumerate(X_test):
                    perturbed_data_point = perturb_data_point(data_point, delta)
                    perturbed_prediction = kdt_.predict(perturbed_data_point)
                    if np.array_equal(predictions[i], perturbed_prediction):
                        robustness_sum += 1
                    perturbed_predictions = np.append(perturbed_predictions, perturbed_prediction)
                all_perturbed_predictions = np.concatenate((all_perturbed_predictions, perturbed_predictions))
                all_y_test = np.concatenate((all_y_test, y_test))
            print("{dataset} _ y_test.shape = ".format(dataset=dataset),all_y_test.shape)

            print("================ Processing ( {dataset} ) dataset, using model = ( {model} )".format(dataset=dataset, model=model), "================")

            evaluate_and_save_results_Hyperparameters(all_y_test, all_predictions, all_original_predictions, all_perturbed_predictions, dataset, baseline="ours", black_model=model, rho=rho_)
print("============================= rho All works are finished! =============================")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

data = pd.read_csv("./ours/Result_Baseline_Hyperparameters.csv")

metrics = ["Fidelity", "precision", "recall", "Robustness"]
parameters = ["max_levels", "n_beams", "rhos", "etas"]
black_models = data["black_model"].unique()

colors = ["red", "blue", "green", "orange", "purple"]
markers = ["o", "s", "D", "^", "v"]

for metric in metrics:
    for i, parameter in enumerate(parameters):
        plt.figure()
        for j, black_model in enumerate(black_models):
            filtered_data = data[data["black_model"] == black_model]
            plt.plot(filtered_data[parameter], filtered_data[metric], color=colors[j], marker=markers[j], label=black_model)
        plt.xlabel(parameter)
        plt.ylabel(metric)
        plt.legend()
        plt.title("{} vs {}".format(metric, parameter))
        
        if not os.path.exists("out_images"):
            os.makedirs("out_images")
        
        plt.savefig("out_images/{}_{}.png".format(parameter, parameter))
        plt.close()
plt.show()


In [ ]:
# Combine all images into a single image
images = []
for metric in metrics:
    rows = []
    for parameter in parameters:
        filenames = ["{}_vs_{}_{}.png".format(metric, parameter, black_model) for black_model in black_models]
        filepaths = [os.path.join("image", filename) for filename in filenames]
        images = [Image.open(filepath) for filepath in filepaths]
        widths, heights = zip(*(i.size for i in images))
        max_height = max(heights)
        total_width = sum(widths)
        combined_image = Image.new("RGB", (total_width, max_height))
        x_offset = 0
        for image in images:
            combined_image.paste(image, (x_offset, 0))
            x_offset += image.size[0]
        rows.append(combined_image)
    widths, heights = zip(*(i.size for i in rows))
    max_width = max(widths)
    total_height = sum(heights)
    combined_row = Image.new("RGB", (max_width, total_height))
    y_offset = 0
    for row in rows:
        combined_row.paste(row, (0, y_offset))
        y_offset += row.size[1]
    images.append(combined_row)

# Save the combined image
combined_image = Image.new("RGB", (combined_row.size[0], total_height))
y_offset = 0
for image in images:
    combined_image.paste(image, (0, y_offset))
    y_offset += image.size[1]
combined_image.save("combined_image.png")